# FLEX

This notebook runs [GOTM](https://gotm.net/) using the [FLEX](https://gotm.net/cases/flex/) test case.

Run the case with four turbulence closure schemes:

- GLS-C01A: The generic length scale ([Umlauf and Burchard, 2003](https://doi.org/10.1357/002224003322005087)) model in the $k$-$\epsilon$ formulation with the weak-equilibrium stability function by [Canuto et al., 2001](https://doi.org/10.1175/1520-0485(2001)031%3C1413:OTPIOP%3E2.0.CO;2) (C01A).

- Three variants of KPP via [CVMix](http://cvmix.github.io):
    - KPP-CVMix ([Large et al., 1994](https://doi.org/10.1029/94RG01872), [Griffies et al., 2015](https://github.com/CVMix/CVMix-description/raw/master/cvmix.pdf))
    - KPPLT-VR12 ([Li et al., 2016](https://doi.org/10.1016%2Fj.ocemod.2015.07.020))
    - KPPLT-LF17 ([Li and Fox-Kemper, 2017](https://doi.org/10.1175%2FJPO-D-17-0085.1))
    
Estimate the Stokes drift from the wind following [Li et al., 2017](https://doi.org/10.1016/j.ocemod.2017.03.016).

In [1]:
import sys
import copy
import numpy as np
import matplotlib.pyplot as plt
# add the path of gotmtool
sys.path.append("../gotmtool")
from gotmtool import *

## Create a model
Create a model with environment file `../../.gotm_env.yaml`, which is created by `gotm_env_init.py`. 

In [2]:
m = Model(name='FLEX', environ='../gotmtool/.gotm_env.yaml')

Take a look at what are defined in the environment file.

In [3]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

   gotmdir_code: /Users/qingli/work/cvmix_in_gotm/gotm/code
   gotmdir_data: /Users/qingli/work/cvmix_in_gotm/gotm/data
  gotmdir_build: /Users/qingli/work/cvmix_in_gotm/gotm/build
    gotmdir_exe: /Users/qingli/work/cvmix_in_gotm/gotm/exe
    gotmdir_run: /Users/qingli/work/cvmix_in_gotm/gotm/run
 gotmdir_figure: /Users/qingli/work/cvmix_in_gotm/gotm/figure
   gotmdir_tool: /Users/qingli/work/cvmix_in_gotm/gotmtool


## Build the model

In [4]:
%%time
m.build()

CPU times: user 2.18 ms, sys: 8.15 ms, total: 10.3 ms
Wall time: 157 ms


## Configuration
Initialize the GOTM configuration

In [5]:
cfg = m.init_config()

Generating default configuration at '/Users/qingli/work/cvmix_in_gotm/gotm/run/FLEX1/gotm.yaml'...
Done!


Update the configuration

In [6]:
# setup
title = 'FLEX'
nlev = 145
cfg['title'] = title
cfg['location']['name'] = 'Flex Experiment 1976'
cfg['location']['latitude'] = 58.92
cfg['location']['longitude'] = 0.32
cfg['location']['depth'] = 145.0
cfg['time']['start'] = '1976-04-06 06:00:00'
cfg['time']['stop']  = '1976-06-07 00:00:00'
cfg['time']['dt']    = 360.0
cfg['grid']['nlev']  = nlev
cfg['grid']['ddu']  = 0.0
cfg['grid']['ddl']  = 0.0

# output
cfg['output']['gotm_out']['title'] = title
cfg['output']['gotm_out']['k1_stop'] = nlev+1
cfg['output']['gotm_out']['k_stop'] = nlev
cfg['output']['gotm_out']['time_unit'] = 'dt'
cfg['output']['gotm_out']['time_step'] = 30

# forcing
datadir = m.environ['gotmdir_data']+'/FLEX'
cfg['temperature']['method'] = 2
cfg['temperature']['file'] = datadir+'/t_prof_file.dat'
cfg['salinity']['method'] = 2
cfg['salinity']['file'] = datadir+'/s_prof_file.dat'
cfg['salinity']['relax']['tau'] = 172800.0
cfg['salinity']['relax']['tau_s'] = 172800.0
cfg['salinity']['relax']['tau_b'] = 172800.0
cfg['surface']['fluxes']['method'] = 2
cfg['surface']['fluxes']['heat']['method'] = 2
cfg['surface']['fluxes']['heat']['file'] = datadir+'/heatflux_file.dat'
cfg['surface']['fluxes']['tx']['method'] = 2
cfg['surface']['fluxes']['tx']['file'] = datadir+'/momentumflux_file.dat'
cfg['surface']['fluxes']['tx']['column'] = 1
cfg['surface']['fluxes']['ty']['method'] = 2
cfg['surface']['fluxes']['ty']['file'] = datadir+'/momentumflux_file.dat'
cfg['surface']['fluxes']['ty']['column'] = 2
cfg['surface']['meteo']['u10']['method'] = 2
cfg['surface']['meteo']['u10']['file'] = datadir+'/meteo.dat'
cfg['surface']['meteo']['u10']['column'] = 1
cfg['surface']['meteo']['v10']['method'] = 2
cfg['surface']['meteo']['v10']['file'] = datadir+'/meteo.dat'
cfg['surface']['meteo']['v10']['column'] = 2
cfg['surface']['meteo']['airp']['method'] = 2
cfg['surface']['meteo']['airp']['file'] = datadir+'/meteo.dat'
cfg['surface']['meteo']['airp']['column'] = 3
cfg['surface']['meteo']['airp']['scale_factor'] = 100.0
cfg['surface']['meteo']['airt']['method'] = 2
cfg['surface']['meteo']['airt']['file'] = datadir+'/meteo.dat'
cfg['surface']['meteo']['airt']['column'] = 4
cfg['surface']['meteo']['hum']['method'] = 2
cfg['surface']['meteo']['hum']['file'] = datadir+'/meteo.dat'
cfg['surface']['meteo']['hum']['column'] = 5
cfg['surface']['meteo']['hum']['type'] = 2
cfg['surface']['meteo']['swr']['method'] = 2
cfg['surface']['meteo']['swr']['file'] = datadir+'/swr_file.dat'
cfg['surface']['longwave_radiation']['method'] = 0
cfg['surface']['longwave_radiation']['file'] = datadir+'/lwr.dat'
cfg['surface']['sst']['method'] = 2
cfg['surface']['sst']['file'] = datadir+'/sst_file.dat'

cfg['stokes_drift']['us']['method'] = 4
cfg['stokes_drift']['vs']['method'] = 4
cfg['stokes_drift']['empirical']['uwnd']['method'] = 2
cfg['stokes_drift']['empirical']['uwnd']['file'] = datadir+'/meteo.dat'
cfg['stokes_drift']['empirical']['uwnd']['column'] = 1
cfg['stokes_drift']['empirical']['vwnd']['method'] = 2
cfg['stokes_drift']['empirical']['vwnd']['file'] = datadir+'/meteo.dat'
cfg['stokes_drift']['empirical']['vwnd']['column'] = 2

# water type
cfg['light_extinction']['method'] = 7
cfg['light_extinction']['A']['method'] = 2
cfg['light_extinction']['A']['file'] = datadir+'/extinct_file.dat'
cfg['light_extinction']['A']['column'] = 1
cfg['light_extinction']['g1']['method'] = 2
cfg['light_extinction']['g1']['file'] = datadir+'/extinct_file.dat'
cfg['light_extinction']['g1']['column'] = 2
cfg['light_extinction']['g2']['method'] = 2
cfg['light_extinction']['g2']['file'] = datadir+'/extinct_file.dat'
cfg['light_extinction']['g2']['column'] = 3

# tidal
cfg['mimic_3d']['ext_pressure']['mode'] = 0
cfg['mimic_3d']['ext_pressure']['dpdx']['method'] = 2
cfg['mimic_3d']['ext_pressure']['dpdx']['file'] = datadir+'/ext_press_file.dat'
cfg['mimic_3d']['ext_pressure']['dpdx']['column'] = 2
cfg['mimic_3d']['ext_pressure']['dpdy']['method'] = 2
cfg['mimic_3d']['ext_pressure']['dpdy']['file'] = datadir+'/ext_press_file.dat'
cfg['mimic_3d']['ext_pressure']['dpdy']['column'] = 3
cfg['mimic_3d']['ext_pressure']['h']['method'] = 2
cfg['mimic_3d']['ext_pressure']['h']['file'] = datadir+'/ext_press_file.dat'
cfg['mimic_3d']['ext_pressure']['h']['column'] = 1

# EOS 
cfg['eq_state']['mode'] = 1
cfg['eq_state']['method'] = 2

# buoyancy
cfg['buoyancy']['NN_ini'] = 1.0e-4


## Run the model

Set the configurations and labels for each run

In [7]:
cfgs = []
labels = []

In [8]:
cfg['turbulence']['turb_method'] = 3
cfg['turbulence']['tke_method'] = 2
cfg['turbulence']['len_scale_method'] = 10
cfg['turbulence']['scnd']['method'] =  2
cfg['turbulence']['scnd']['scnd_coeff'] =  5
cfg['turbulence']['turb_param']['length_lim'] = 'false'
cfg['turbulence']['turb_param']['compute_c3'] = 'true'
cfg['turbulence']['turb_param']['Ri_st'] = 0.25
cfg['turbulence']['generic']['gen_m'] = 1.5 
cfg['turbulence']['generic']['gen_n'] = -1.0
cfg['turbulence']['generic']['gen_p'] = 3.0 
cfg['turbulence']['generic']['cpsi1'] = 1.44
cfg['turbulence']['generic']['cpsi2'] = 1.92
cfg['turbulence']['generic']['cpsi3minus'] = -0.63
cfg['turbulence']['generic']['cpsi3plus'] = 1.0 
cfg['turbulence']['generic']['sig_kpsi'] = 1.0 
cfg['turbulence']['generic']['sig_psi'] = 1.3
cfgs.append(copy.deepcopy(cfg))
labels.append('GLS-C01A_tidal')

In [9]:
# turn tidal forcing off
cfg['mimic_3d']['ext_pressure']['dpdx']['method'] = 0
cfg['mimic_3d']['ext_pressure']['dpdy']['method'] = 0
cfg['mimic_3d']['ext_pressure']['h']['method'] = 0
cfgs.append(copy.deepcopy(cfg))
labels.append('GLS-C01A')

In [10]:
cfg['turbulence']['turb_method'] = 100
cfgs.append(copy.deepcopy(cfg))
labels.append('KPP-CVMix')

In [11]:
cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 1
cfgs.append(copy.deepcopy(cfg))
labels.append('KPPLT-VR12')

In [12]:
cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 2
cfgs.append(copy.deepcopy(cfg))
labels.append('KPPLT-LF17')

Run the cases in parallel with 2 processes 

In [13]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, nproc=2)

CPU times: user 17.1 ms, sys: 13.3 ms, total: 30.5 ms
Wall time: 7.6 s
